### Importing required libraries

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras

### Reading the data

In [ ]:
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.cifar10.load_data()
train_imgs = train_imgs/ 255.0
test_imgs = test_imgs/ 255.0

num_classes = 10
train_lbls = np.squeeze(keras.utils.to_categorical(train_lbls, num_classes))
test_lbls = np.squeeze(keras.utils.to_categorical(test_lbls, num_classes))

### Dispaly of data

In [ ]:
img = (train_imgs[400].squeeze())
plt.imshow(img, cmap='gray')

### Creating model 'BK'

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape= (32,32,3)),
    tf.keras.layers.Conv2D(32,5, strides = (1, 1), padding='same', use_bias=True,activation = 'relu'),
    tf.keras.layers.MaxPool2D((2, 2),(2, 2)),
    tf.keras.layers.Conv2D(32, 5, strides = (1, 1), padding='same', use_bias=True,activation = 'relu'),
    tf.keras.layers.GlobalMaxPooling2D(data_format='channels_last'),
    tf.keras.layers.Dense(10, activation = 'sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
])
model.summary()

### Initializing parameters for training

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import callbacks

def step_decay(epoch):
    initial_lr=0.001
    decay_rate = 0.01
    decay_step = 30
    lrate = initial_lr * decay_rate ** (epoch/decay_step)
    print("learning_rate")
    print(lrate)
    return lrate

Adam = tf.keras.optimizers.Adam(lr=0.0)
model.compile(loss='categorical_crossentropy', optimizer = Adam, metrics=['accuracy'])

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

### Compiling the model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = Adam, metrics=['accuracy'])

### Fit the model

In [ ]:
#With Noise
history = model.fit(train_imgs, train_lbls, batch_size=128, epochs=100, validation_split=0.1, callbacks = callbacks_list)

In [ ]:
print(history.history.keys())

### Display learning curve

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Test the model

In [ ]:
#Without Noise
model.evaluate(test_imgs, test_lbls)

#With Noise
model.evaluate(noise_test_imgs, test_lbls)

### Predictions of the model

In [ ]:
predictions = model.predict(test_imgs)

In [ ]:
n = 4
plt.figure(figsize=(15,15))
for i in range(n):
    plt.subplot(1,n,i+1)
    plt.imshow(test_imgs[i])
    plt.title("Lable:{}\nPredicted:{}".format(test_lbls[i],np.argmax(predictions[i])))
    plt.axis='off'
plt.show()

### Saving the model and the predictions

In [ ]:
model.save_weights('Model_BK.h5') #Depending on the level of noise and the model, name is given

In [ ]:
model_predictions = []
ground_truth_values = []
for i in range(len(test_imgs)):
    model_predictions.append(np.argmax(predictions[i]))
      ground_truth_values.append(np.argmax(test_lbls[i]))


correct_prediction = [i if i==j else 0 for i, j in zip(model_predictions, ground_truth_values)]
import pandas as pd
prediction_df = pd.DataFrame(correct_prediction, columns=['result']).to_csv('BK_1_prediction.csv', index=False)